In [3]:
import pandas as pd, math, numpy as np
import cirq
import mat2qubit as m2q
from openfermion import QubitOperator
from itertools import combinations
import import_ipynb
import Hermite_enc

importing Jupyter notebook from Hermite_enc.ipynb
importing Jupyter notebook from Hermite_helper_fns.ipynb


### Test: Adding Hermite representation to the mat2qubit pipeline

Below, we try to construct the Hermite representation of the AMV model

$$ H = \frac{1}{2}\Sigma_{k}^{M}\left(q_k^2+p_k^2\right) + \Sigma_{ijk}k_{ijk}q_iq_jq_k+ \Sigma_{ijkl}h_{ijkl}q_iq_jq_kq_l
$$

References:
1. Sawaya and Huh, J. Phys. Chem. Lett. 2019, 10, 13, 3586–3591.
2. Smith, Liu, and Noid. Chemical Physics, 1984, 89(3), 345– 351.

In [4]:
def create_AMV_Hamiltonian(M, d_values, freq, k1, k2):
    harmonic_terms = m2q.qSymbOp("")
    for i in range(len(freq)):
        harmonic_terms += m2q.qSymbOp(f"({freq[i]}) [q2_{i}] ++ ({freq[i]}) [p2_{i}]")
    
    anharmonic_terms_1 = m2q.qSymbOp("")
    anharmonic_terms_2 = m2q.qSymbOp("")
    for i in range(len(k1)):
        for j in range(len(k1[i])):
            for l in range(len(k1[i][j])):
                if k1[i][j][l] != 0:
                    aux_term_1 = m2q.qSymbOp("")
                    aux_term_1 += m2q.qSymbOp(f"({k1[i][j][l]}) [q_{i} q_{j} q_{l}]")
                    aux_term_1.simplifyQuadExponents()
                    anharmonic_terms_1 += aux_term_1
                    #print(f"{k1[i][j][l]} [q_{i} q_{j} q_{l}]\n")
    
    for i in range(len(k2)):
        for j in range(len(k2[i])):
            for l in range(len(k2[i][j])):
                for h in range(len(k2[i][j][l])):
                    if k2[i][j][l][h] !=0:
                        aux_term_2 = m2q.qSymbOp("")
                        aux_term_2 = m2q.qSymbOp(f"({k2[i][j][l][h]}) [q_{i} q_{j} q_{l} q_{h}]")
                        aux_term_2.simplifyQuadExponents()
                        anharmonic_terms_2  += aux_term_2
                        #print(f"{k2[i][j][l][h]} [q_{i} q_{j} q_{l} q_{h}]\n")
                        
                    
    anharmonic_terms_orig = m2q.qSymbOp('''44 [q3_0] ++ -19 [q2_0 q_1] ++ -12 [q_0 q2_1] 
    ++ 159 [q_0 q2_2] ++ -7.0 [q3_1] ++ 4.7 [q_1 q2_2] ++ 1.8 [q4_0] ++ -3.1 [q2_0 q2_1] 
    ++ 15 [q2_0 q2_2] ++ -1.4 [q4_1] ++ -6.5 [q2_1 q2_2] ++ 3.0 [q4_2]''')
                
    
        
    
    print("\nSymbolic Hamiltonian")
    symb_hamiltonian = harmonic_terms + anharmonic_terms_1 + anharmonic_terms_2
    #symb_hamiltonian = harmonic_terms + anharmonic_terms_orig
    print(symb_hamiltonian,"\n\n")
    # print(str(symb_hamiltonian))
    # print("Type: ",type(symb_hamiltonian))

    
    #print("\nQubit Operator [Gray code; d=(4,8,4)]")
    ssid_order = [str(i) for i in range(M)] 
    #ssid_order = ('0','1','2','3')
    AMV_dlev_obj_SB = m2q.symbop_to_dlevcompositeop(symb_hamiltonian, ssid_order,d_values,'stdbinary')
    AMV_dlev_obj_Gray = m2q.symbop_to_dlevcompositeop(symb_hamiltonian, ssid_order,d_values,'gray')
    AMV_dlev_obj_Unary = m2q.symbop_to_dlevcompositeop(symb_hamiltonian, ssid_order,d_values,'unary')
    #AMV_dlev_obj_BU = m2q.symbop_to_dlevcompositeop(symb_hamiltonian, ssid_order,d_values,'bu')
    
    AMV_qub_op_SB = AMV_dlev_obj_SB.opToPauli()
    AMV_qub_op_Gray = AMV_dlev_obj_Gray.opToPauli()
    AMV_qub_op_Unary = AMV_dlev_obj_Unary.opToPauli()
    #AMV_qub_op_BU = AMV_dlev_obj_BU.opToPauli()
    
    new_d_list=[]
    for i in d_values:
        new_d_list.append(i)
    ham_qub_op_Hermite = Hermite_enc.Hermite_encoding(symb_hamiltonian,new_d_list)[0]
    
    
    return AMV_qub_op_SB, AMV_qub_op_Gray, AMV_qub_op_Unary, ham_qub_op_Hermite

In [5]:
M=3
d_val=(4,8,4)
f=(1171,525,1378)
K1=( ((44,-19,0),(0,-12,0),(0,0,159)),   
   ((0,0,0),(0,-7,0),(0,0,4.7)),
   ((0,0,0),(0,0,0),(0,0,0))
   )
K2_0 =( ((1.8,0,0),(0,-3.1,0),(0,0,15)),
        ((0,0,0),(0,0,0),(0,0,0)),
        ((0,0,0),(0,0,0),(0,0,0)) )
K2_1 =( ((0,0,0),(0,0,0),(0,0,0)),
        ((0,0,0),(0,-1.4,0),(0,0,-6.5)),
        ((0,0,0),(0,0,0),(0,0,0)) )
K2_2 =( ((0,0,0),(0,0,0),(0,0,0)),
        ((0,0,0),(0,0,0),(0,0,0)),
        ((0,0,0),(0,0,0),(0,0,3.0)) )

K2 = (K2_0, K2_1, K2_2)
    
    

AMV_SB, AMV_Gray, AMV_Unary, AMV_Hermite = create_AMV_Hamiltonian(M,d_val,f,K1,K2)
# print("\n Anharmonic Molecular Vibrational QubitOperator (Display Gray)\n\n", AMV_Gray)
print("\n Anharmonic Molecular Vibrational QubitOperator (Display Hermite)\n\n", AMV_Hermite)


Symbolic Hamiltonian
((1171+0j)) [q2_0]
++ ((1171+0j)) [p2_0]
++ ((525+0j)) [q2_1]
++ ((525+0j)) [p2_1]
++ ((1378+0j)) [q2_2]
++ ((1378+0j)) [p2_2]
++ ((44+0j)) [q3_0]
++ ((-19+0j)) [q2_0 q_1]
++ ((-12+0j)) [q_0 q2_1]
++ ((159+0j)) [q_0 q2_2]
++ ((-7+0j)) [q3_1]
++ ((4.7+0j)) [q_1 q2_2]
++ ((1.8+0j)) [q4_0]
++ ((-3.1+0j)) [q2_0 q2_1]
++ ((15+0j)) [q2_0 q2_2]
++ ((-1.4+0j)) [q4_1]
++ ((-6.5+0j)) [q2_1 q2_2]
++ ((3+0j)) [q4_2] 



 Anharmonic Molecular Vibrational QubitOperator (Display Hermite)

 (-2121445+0j) [] +
(543+0j) [Z0] +
(12387.6+0j) [Z0 Z1] +
(-76+0j) [Z0 Z1 Z2] +
(-49.6+0j) [Z0 Z1 Z2 Z3] +
(-99.2+0j) [Z0 Z1 Z2 Z4] +
(-152+0j) [Z0 Z1 Z3] +
(-198.4+0j) [Z0 Z1 Z3 Z4] +
(-304+0j) [Z0 Z1 Z4] +
(240+0j) [Z0 Z1 Z5 Z6] +
(-48+0j) [Z0 Z2 Z3] +
(-96+0j) [Z0 Z2 Z4] +
(-192+0j) [Z0 Z3 Z4] +
(636+0j) [Z0 Z5 Z6] +
(1086+0j) [Z1] +
(-96+0j) [Z1 Z2 Z3] +
(-192+0j) [Z1 Z2 Z4] +
(-384+0j) [Z1 Z3 Z4] +
(1272+0j) [Z1 Z5 Z6] +
(-71.5+0j) [Z2] +
(-1879725.5999999999+0j) [Z2 Z3] +
(-104+0j) [Z2 Z

In [6]:
def create_representations(op_list,d,ind):
    '''
    Produce the various encoding representations of a Hamiltonian operator
    
    Args:
    op_list: list containing the operator strings to be considered
                e.g. input ['q2_i','q_j'] in order to consider '[q2_i q_j]'
    d: a list containing the d-level of each site/register/particle
    ind: indicate what kind of term is given:
            0: pure X
            1: pure P
            2: XP
            3: PX
    
    '''
    ham_model = m2q.qSymbOp("")
    
    op_str = ""
    for i in range(len(op_list)):
        if op_str=="" and i==len(op_list)-1:
            op_str+="["+op_list[i]+"]"
        elif op_str=="" and i<len(op_list)-1:
            op_str+="["+op_list[i]
        elif i<(len(op_list)-1):
            op_str+= " " + op_list[i]
        elif i==len(op_list)-1:
             op_str+= " " + op_list[i] + "]"
            

    ham_model += m2q.qSymbOp(op_str)
    # print("Display symbolic Hamiltonian:\n\n",ham_model)
    
    
    q = len(d)
    ssid_order = [str(i) for i in range(q)] 
    
    ham_dlev_obj_SB = m2q.symbop_to_dlevcompositeop(ham_model, ssid_order,d,'stdbinary')
    ham_dlev_obj_Gray = m2q.symbop_to_dlevcompositeop(ham_model, ssid_order,d,'gray')
    ham_dlev_obj_unary = m2q.symbop_to_dlevcompositeop(ham_model, ssid_order,d,'unary')
    
    #print("\nConvert to QubitOperator (Display SB)")
    ham_qub_op_SB = ham_dlev_obj_SB.opToPauli()
    ham_qub_op_Gray = ham_dlev_obj_Gray.opToPauli()
    ham_qub_op_unary = ham_dlev_obj_unary.opToPauli()
    
    # print(ham_model)
    ham_qub_op_Hermite = Hermite_enc.Hermite_encoding(ham_model,d)[ind]
    
    return ham_qub_op_SB, ham_qub_op_Gray, ham_qub_op_unary, ham_qub_op_Hermite

In [7]:
test = create_representations(["q_0","q_1"],[4,4],0)
print("\nDisplay Hermite representation:\n\n",test[3], "\n")
print("\nDisplay charge encoding (SB) representation:\n\n",test[0])


Display Hermite representation:

 (1+0j) [Z0 Z2] +
(2+0j) [Z0 Z3] +
(2+0j) [Z1 Z2] +
(4+0j) [Z1 Z3] 


Display charge encoding (SB) representation:

 0.4829629131445341 [X0 X1 X2] +
0.25 [X0 X1 X2 X3] +
-0.12940952255126037 [X0 X1 X2 Z3] +
(0.25+0j) [X0 X1 Y2 Y3] +
-0.24999999999999994 [X0 Z1 X2] +
-0.12940952255126037 [X0 Z1 X2 X3] +
0.06698729810778066 [X0 Z1 X2 Z3] +
(-0.12940952255126037+0j) [X0 Z1 Y2 Y3] +
0.9330127018922192 [X0 X2] +
0.4829629131445341 [X0 X2 X3] +
-0.24999999999999994 [X0 X2 Z3] +
(0.4829629131445341+0j) [X0 Y2 Y3] +
(0.4829629131445341+0j) [Y0 Y1 X2] +
(0.25+0j) [Y0 Y1 X2 X3] +
(-0.12940952255126037+0j) [Y0 Y1 X2 Z3] +
(0.25+0j) [Y0 Y1 Y2 Y3]


In [8]:
for i in combinations('ABCD', 2):
    print(i)
    # print(i[0]+i[1])

print('\n')
for j in combinations('123',2):
    print(j)

('A', 'B')
('A', 'C')
('A', 'D')
('B', 'C')
('B', 'D')
('C', 'D')


('1', '2')
('1', '3')
('2', '3')


In [9]:
def dataframe_of_operators():
    '''
    Produce a data frame containing common operators along with their representations
    in charge encodings (SB,Gray,Unary) and Hermite encoding.
    '''
    
    encodings=["Hermite","Charge Encoding SB","Charge Encoding Gray","Charge Encoding Unary" ]
    
    d_level_list=[4,8,16,32,64,128]
    single_d_levels=[]
    two_d_levels=[]
    three_d_levels=[]
    four_d_levels=[]
    five_d_levels=[]
    six_d_levels=[]
    
    single_x_op_list=[["q_0"],["q2_0"],["q3_0"],["q4_0"],["q5_0"],["q6_0"]]
    single_p_op_list=[["p_0"],["p2_0"]]
    
    two_op_x_list=[["q_0","q_1"],["q2_0","q2_1"],["q2_0","q_1"]]
    two_op_p_list=[["p_0","p_1"],["p2_0","p2_1"],["p2_0","p_1"]]
    
    three_op_x_list=[["q_0","q_1","q_2"],["q2_0","q2_1","q2_2"],["q2_0","q_1","q_2"],["q2_0","q2_1","q_2"]]
    
    four_op_x_list=[["q_0","q_1","q_2","q_3"],["q2_0","q2_1","q2_2","q2_3"],
                    ["q2_0","q_1","q_2","q_3"],["q2_0","q2_1","q_2","q_3"],["q2_0","q2_1","q2_2","q_3"]]
    
    five_op_x_list=[["q_0","q_1","q_2","q_3","q_4"],["q2_0","q2_1","q2_2","q2_3","q2_4"],
                    ["q2_0","q_1","q_2","q_3","q_4"],["q2_0","q2_1","q_2","q_3","q_4"],
                    ["q2_0","q2_1","q2_2","q_3","q_4"],["q2_0","q2_1","q2_2","q2_3","q_4"]]
    
    six_op_x_list=[["q_0","q_1","q_2","q_3","q_4","q_5"],["q2_0","q2_1","q2_2","q2_3","q2_4","q2_5"],
                    ["q2_0","q_1","q_2","q_3","q_4","q_5"],["q2_0","q2_1","q_2","q_3","q_4","q_5"],
                    ["q2_0","q2_1","q2_2","q_3","q_4","q_5"],["q2_0","q2_1","q2_2","q2_3","q_4","q_5"],
                    ["q2_0","q2_1","q2_2","q2_3","q2_4","q_5"]]
    
    for i in d_level_list:
        single_d_levels.append([i])
        two_d_levels.append([i,i])
        three_d_levels.append([i,i,i])
        four_d_levels.append([i,i,i,i])
        five_d_levels.append([i,i,i,i,i])
        six_d_levels.append([i,i,i,i,i,i])
    
    
    
    # Define the column titles
    columns = ['Operator', 'd-level', 'encoding_type', 'Qubit_representation']

    # Create an empty DataFrame with the column titles
    df = pd.DataFrame(columns=columns)
    
    # Read the file into a DataFrame
    # existing_df = pd.read_csv('op_df.csv')
    
    # Define the data for the new row
    for d in single_d_levels:    
        for op_list in single_x_op_list:
            op=""
            for term in op_list:
                if op=="":
                    op+=term
                else:
                    op+= " " + term
                
            reps = create_representations(op_list,d,0)
            for enc in encodings:  
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                
                new_row = {
                            'Operator': op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                print("Completed: ["+op+", "+enc+", "+str(d)+"]")
                
        for p_op_list in single_p_op_list:
            p_op=""
            for term in p_op_list:
                if p_op=="":
                    p_op+=term
                else:
                    p_op+= " " + term
            
            reps = create_representations(p_op_list,d,1)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': p_op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                print("Completed: ["+p_op+", "+enc+", "+str(d)+"]")
                
                
    
    for d in two_d_levels:
        for op_list in two_op_x_list:
            op=""
            for term in op_list:
                if op=="":
                    op+=term
                else:
                    op+= " " + term
            
            reps = create_representations(op_list,d,0)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                print("Completed: ["+op+", "+enc+", "+str(d)+"]")
                
        for p_op_list in two_op_p_list:
            p_op=""
            for term in p_op_list:
                if p_op=="":
                    p_op+=term
                else:
                    p_op+= " " + term
            
            reps = create_representations(p_op_list,d,1)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': p_op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                print("Completed: ["+p_op+", "+enc+", "+str(d)+"]")
                
                
    
    for d in three_d_levels:
        for op_list in three_op_x_list:
            op=""
            for term in op_list:
                if op=="":
                    op+=term
                else:
                    op+= " " + term
            
            reps = create_representations(op_list,d,0)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                print("Completed: ["+op+", "+enc+", "+str(d)+"]")
                
#     for d in four_d_levels:
#         for op_list in four_op_x_list:
#             op=""
#             for term in op_list:
#                 if op=="":
#                     op+=term
#                 else:
#                     op+= " " + term
            
#             reps = create_representations(op_list,d,0)
            
#             for enc in encodings:
#                 if enc=="Hermite":
#                     qub_rep=reps[3]
#                 elif enc=="Charge Encoding SB":
#                     qub_rep=reps[0]
#                 elif enc=="Charge Encoding Gray":
#                     qub_rep=reps[1]
#                 elif enc=="Charge Encoding Unary":
#                     qub_rep=reps[2]
                
#                 new_row = {
#                             'Operator': op,
#                             'd-level': d,
#                             'encoding_type': enc,
#                             'Qubit_representation': qub_rep 
#                             }
            
#                 # Convert the new_row dictionary into a DataFrame
#                 new_row_df = pd.DataFrame([new_row])
            
#                 # Concatenate the existing DataFrame with the new row DataFrame
#                 df = pd.concat([df, new_row_df], ignore_index=True)
#                 print("Completed: ["+op+", "+enc+", "+str(d)+"]")
                
        
    return df
        
        
    

In [10]:
op_df=dataframe_of_operators()
op_df.to_csv('op_df.csv',index=False)




Completed: [q_0, Hermite, [4]]
Completed: [q_0, Charge Encoding SB, [4]]
Completed: [q_0, Charge Encoding Gray, [4]]
Completed: [q_0, Charge Encoding Unary, [4]]
Completed: [q2_0, Hermite, [4]]
Completed: [q2_0, Charge Encoding SB, [4]]
Completed: [q2_0, Charge Encoding Gray, [4]]
Completed: [q2_0, Charge Encoding Unary, [4]]
Completed: [q3_0, Hermite, [4]]
Completed: [q3_0, Charge Encoding SB, [4]]
Completed: [q3_0, Charge Encoding Gray, [4]]
Completed: [q3_0, Charge Encoding Unary, [4]]
Completed: [q4_0, Hermite, [4]]
Completed: [q4_0, Charge Encoding SB, [4]]
Completed: [q4_0, Charge Encoding Gray, [4]]
Completed: [q4_0, Charge Encoding Unary, [4]]
Completed: [q5_0, Hermite, [4]]
Completed: [q5_0, Charge Encoding SB, [4]]
Completed: [q5_0, Charge Encoding Gray, [4]]
Completed: [q5_0, Charge Encoding Unary, [4]]
Completed: [q6_0, Hermite, [4]]
Completed: [q6_0, Charge Encoding SB, [4]]
Completed: [q6_0, Charge Encoding Gray, [4]]
Completed: [q6_0, Charge Encoding Unary, [4]]
Complete

KeyboardInterrupt: 

In [11]:
print("DONE!")

DONE!


In [15]:
def create_single_op_df():
    '''
    Produce a data frame containing common single operators along with their representations
    in charge encodings (SB,Gray,Unary) and Hermite encoding.
    '''
    
    encodings=["Hermite","Charge Encoding SB","Charge Encoding Gray","Charge Encoding Unary" ]
    
    d_level_list=[4,8,16,32,64,128]
    single_d_levels=[]
    
    
    single_x_op_list=[["q_0"],["q2_0"],["q3_0"],["q4_0"],["q5_0"],["q6_0"]]
    single_p_op_list=[["p_0"],["p2_0"]]
    
    for i in d_level_list:
        single_d_levels.append([i])
        
    
    
    
    # Define the column titles
    columns = ['Operator', 'd-level', 'encoding_type', 'Qubit_representation']

    # Create an empty DataFrame with the column titles
    single_op_df = pd.DataFrame(columns=columns)
    
    # Read the file into a DataFrame
    # existing_df = pd.read_csv('op_df.csv')
    
    # Define the data for the new row
    for d in single_d_levels:    
        for op_list in single_x_op_list:
            op=""
            for term in op_list:
                if op=="":
                    op+=term
                else:
                    op+= " " + term
                
            reps = create_representations(op_list,d,0)
            for enc in encodings:  
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                
                new_row = {
                            'Operator': op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                # df = pd.concat([df, new_row_df], ignore_index=True)
                single_op_df = pd.concat([single_op_df, new_row_df], ignore_index=True)
                print("Completed: ["+op+", "+enc+", "+str(d)+"]")
                
        for p_op_list in single_p_op_list:
            p_op=""
            for term in p_op_list:
                if p_op=="":
                    p_op+=term
                else:
                    p_op+= " " + term
            
            reps = create_representations(p_op_list,d,1)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': p_op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                # df = pd.concat([df, new_row_df], ignore_index=True)
                single_op_df = pd.concat([single_op_df, new_row_df], ignore_index=True)
                print("Completed: ["+p_op+", "+enc+", "+str(d)+"]")
                
        
    return single_op_df

In [16]:
single_op_df=create_single_op_df()
single_op_df.to_csv('single_op_df.csv',index=False)

Completed: [q_0, Hermite, [4]]
Completed: [q_0, Charge Encoding SB, [4]]
Completed: [q_0, Charge Encoding Gray, [4]]
Completed: [q_0, Charge Encoding Unary, [4]]
Completed: [q2_0, Hermite, [4]]
Completed: [q2_0, Charge Encoding SB, [4]]
Completed: [q2_0, Charge Encoding Gray, [4]]
Completed: [q2_0, Charge Encoding Unary, [4]]
Completed: [q3_0, Hermite, [4]]
Completed: [q3_0, Charge Encoding SB, [4]]
Completed: [q3_0, Charge Encoding Gray, [4]]
Completed: [q3_0, Charge Encoding Unary, [4]]
Completed: [q4_0, Hermite, [4]]
Completed: [q4_0, Charge Encoding SB, [4]]
Completed: [q4_0, Charge Encoding Gray, [4]]
Completed: [q4_0, Charge Encoding Unary, [4]]
Completed: [q5_0, Hermite, [4]]
Completed: [q5_0, Charge Encoding SB, [4]]
Completed: [q5_0, Charge Encoding Gray, [4]]
Completed: [q5_0, Charge Encoding Unary, [4]]
Completed: [q6_0, Hermite, [4]]
Completed: [q6_0, Charge Encoding SB, [4]]
Completed: [q6_0, Charge Encoding Gray, [4]]
Completed: [q6_0, Charge Encoding Unary, [4]]
Complete

In [ ]:
def create_double_op_df():
    '''
    Produce a data frame containing common double operators along with their representations
    in charge encodings (SB,Gray,Unary) and Hermite encoding.
    '''
    
    encodings=["Hermite","Charge Encoding SB","Charge Encoding Gray","Charge Encoding Unary" ]
    
    d_level_list=[4,8,16,32,64,128]
    two_d_levels=[]
    
    for i in d_level_list:
        two_d_levels.append([i,i])
        
    two_op_x_list=[["q_0","q_1"],["q2_0","q2_1"],["q2_0","q_1"]]
    two_op_p_list=[["p_0","p_1"],["p2_0","p2_1"],["p2_0","p_1"]]
    
    
    # Define the column titles
    columns = ['Operator', 'd-level', 'encoding_type', 'Qubit_representation']

    # Create an empty DataFrame with the column titles
    double_op_df = pd.DataFrame(columns=columns)
    
    # Read the file into a DataFrame
    # existing_df = pd.read_csv('op_df.csv')
    
    # Define the data for the new row
    
    for d in two_d_levels:
        for op_list in two_op_x_list:
            op=""
            for term in op_list:
                if op=="":
                    op+=term
                else:
                    op+= " " + term
            
            reps = create_representations(op_list,d,0)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                double_op_df = pd.concat([double_op_df, new_row_df], ignore_index=True)
                print("Completed: ["+op+", "+enc+", "+str(d)+"]")
                
        for p_op_list in two_op_p_list:
            p_op=""
            for term in p_op_list:
                if p_op=="":
                    p_op+=term
                else:
                    p_op+= " " + term
            
            reps = create_representations(p_op_list,d,1)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': p_op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                double_op_df = pd.concat([double_op_df, new_row_df], ignore_index=True)
                print("Completed: ["+p_op+", "+enc+", "+str(d)+"]")
                
                
    
        
    return double_op_df
        
        
    

In [ ]:
double_op_df=create_double_op_df()
double_op_df.to_csv('double_op_df.csv',index=False)

In [ ]:
def create_triple_op_df():
    '''
    Produce a data frame containing  triple operators along with their representations
    in charge encodings (SB,Gray,Unary) and Hermite encoding.
    '''
    
    encodings=["Hermite","Charge Encoding SB","Charge Encoding Gray","Charge Encoding Unary" ]
    
    d_level_list=[4,8,16,32,64,128]
    three_d_levels=[]
    
    three_op_x_list=[["q_0","q_1","q_2"],["q2_0","q2_1","q2_2"],["q2_0","q_1","q_2"],["q2_0","q2_1","q_2"]]
    
    for i in d_level_list:
        three_d_levels.append([i,i,i])
    
    # Define the column titles
    columns = ['Operator', 'd-level', 'encoding_type', 'Qubit_representation']

    # Create an empty DataFrame with the column titles
    triple_op_df = pd.DataFrame(columns=columns)
    
    # Read the file into a DataFrame
    # existing_df = pd.read_csv('op_df.csv')
    
    # Define the data for the new row
    for d in three_d_levels:
        for op_list in three_op_x_list:
            op=""
            for term in op_list:
                if op=="":
                    op+=term
                else:
                    op+= " " + term
            
            reps = create_representations(op_list,d,0)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                triple_op_df = pd.concat([triple_op_df, new_row_df], ignore_index=True)
                print("Completed: ["+op+", "+enc+", "+str(d)+"]")

    return triple_op_df
        
        
    

In [ ]:
triple_op_df=create_triple_op_df()
triple_op_df.to_csv('triple_op_df.csv',index=False)

In [ ]:
def create_quad_op_df():
    '''
    Produce a data frame containing common quad operators along with their representations
    in charge encodings (SB,Gray,Unary) and Hermite encoding.
    '''
    
    encodings=["Hermite","Charge Encoding SB","Charge Encoding Gray","Charge Encoding Unary" ]
    
    d_level_list=[4,8,16,32,64,128]
    four_d_levels=[]
    
    four_op_x_list=[["q_0","q_1","q_2","q_3"],["q2_0","q2_1","q2_2","q2_3"],
                    ["q2_0","q_1","q_2","q_3"],["q2_0","q2_1","q_2","q_3"],["q2_0","q2_1","q2_2","q_3"]]

    for i in d_level_list:
        four_d_levels.append([i,i,i,i])
    
    
    # Define the column titles
    columns = ['Operator', 'd-level', 'encoding_type', 'Qubit_representation']

    # Create an empty DataFrame with the column titles
    quad_op_df = pd.DataFrame(columns=columns)
    
    # Read the file into a DataFrame
    # existing_df = pd.read_csv('op_df.csv')
    
    # Define the data for the new row             
    for d in four_d_levels:
        for op_list in four_op_x_list:
            op=""
            for term in op_list:
                if op=="":
                    op+=term
                else:
                    op+= " " + term
            
            reps = create_representations(op_list,d,0)
            
            for enc in encodings:
                if enc=="Hermite":
                    qub_rep=reps[3]
                elif enc=="Charge Encoding SB":
                    qub_rep=reps[0]
                elif enc=="Charge Encoding Gray":
                    qub_rep=reps[1]
                elif enc=="Charge Encoding Unary":
                    qub_rep=reps[2]
                
                new_row = {
                            'Operator': op,
                            'd-level': d,
                            'encoding_type': enc,
                            'Qubit_representation': qub_rep 
                            }
            
                # Convert the new_row dictionary into a DataFrame
                new_row_df = pd.DataFrame([new_row])
            
                # Concatenate the existing DataFrame with the new row DataFrame
                df = pd.concat([df, new_row_df], ignore_index=True)
                quad_op_df = pd.concat([quad_op_df, new_row_df], ignore_index=True)
                print("Completed: ["+op+", "+enc+", "+str(d)+"]")
                
        
    return quad_op_df
        
        
    

In [ ]:
quad_op_df=create_quad_op_df()
quad_op_df.to_csv('quad_op_df.csv',index=False)